In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import os
import dill
import warnings

import time as time
import pandas as pd
import numpy as np

from glob import glob

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [3]:
# To Read out 100 rows rather than the default value. 
pd.set_option('display.max_rows', 1000)

# Not prinitng the warnings
warnings.filterwarnings('ignore')

# Setting the working path for data input and result outputs
os.chdir('D:\\Spring 2019\\DS 440\\Data')

In [4]:
#filenames = glob('D:\Spring 2019\DS 440\Data\kplr_dr25_inj*.csv')
#df = pd.concat([pd.read_csv(f) for f in filenames], ignore_index = True)

# Problem 2
#df = pd.read_csv('kplr_dr25_inj1_plti.csv', header = 0)

# Problem 2
df = pd.read_csv('kplr_dr25_inj1_tces.csv', header = 0)

print('Size: ')
print(df.shape)

Size: 
(45377, 26)


In [5]:
cols = ['TCE_ID', 'KIC', 'Disp', 'Score', 'period', 'epoch', 'NTL', 'SS', 
        'CO', 'EM', 'Expected_MES', 'MES', 'NTran', 'depth', 'duration', 'Rp',
        'Rs', 'Ts', 'logg', 'a', 'Rp/Rs', 'a/Rs', 'impact', 'SNR_DV', 'Sp',
        'Fit_Prov']
df = df[cols]
df.columns

Index(['TCE_ID', 'KIC', 'Disp', 'Score', 'period', 'epoch', 'NTL', 'SS', 'CO',
       'EM', 'Expected_MES', 'MES', 'NTran', 'depth', 'duration', 'Rp', 'Rs',
       'Ts', 'logg', 'a', 'Rp/Rs', 'a/Rs', 'impact', 'SNR_DV', 'Sp',
       'Fit_Prov'],
      dtype='object')

In [6]:
'''
temp_df = df.iloc[:, 0:15]
df_drop = temp_df[temp_df.isnull().any(axis=1)]
temp_df = temp_df.drop(df_drop.index.values)
print(temp_df.shape)

X = temp_df.iloc[:, 1:14]
Y = temp_df.iloc[:, 14]
'''

'\ntemp_df = df.iloc[:, 0:15]\ndf_drop = temp_df[temp_df.isnull().any(axis=1)]\ntemp_df = temp_df.drop(df_drop.index.values)\nprint(temp_df.shape)\n\nX = temp_df.iloc[:, 1:14]\nY = temp_df.iloc[:, 14]\n'

In [7]:
df['Disp'] = df['Disp'].replace('PC', 1)
df['Disp'] = df['Disp'].replace('FP', 0)

X = df.iloc[:, 6:25]
Y = df.iloc[:, 2]

In [8]:
abc = AdaBoostClassifier() #n_estimators
etc = ExtraTreesClassifier() #n_estimators, max_depth, min_samples_split, min_samples_leaf
rfc = RandomForestClassifier() #n_estimators, max_depth, min_samples_split, min_samples_leaf
gbc = GradientBoostingClassifier() #n_estimators, max_depth, min_samples_split, min_samples_leaf

xgb = XGBClassifier() #n_estimators, max_depth
svc = SVC() #C, kernel, gamma, degree
dtc = DecisionTreeClassifier() #max_depth, min_samples_split, min_samples_leaf
mlp = MLPClassifier()
knc = KNeighborsClassifier() #n_neighbors, p
gpc = GaussianProcessClassifier()

kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 7)

In [9]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 350, num = 5)]

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(start = 2, stop = 30, num = 10)]

# Minimum number of samples required to split a node
min_samples_split = [2, 4, 6, 8, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [0.10, 0.25, 0.50, 1, 2, 4]

# Minimum number of neighbors required for each node
n_neighbors = [int(x) for x in np.linspace(start = 3, stop = 30, num = 10)]

# Minimum Power parameter required for the Minkowski metric
p = [float(x) for x in np.linspace(start = 1, stop = 5, num = 10)]

# Minimum penalty parameter C of the error term
C = [0.001, 0.01, 0.1, 1, 10]

# Type of kernel algorithm
kernel = ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']

# Minimum kernel coefficient
gamma = [0.001, 0.01, 0.1, 1, 10]

# Degree of the polynomial kernel function
degree = [0, 1, 2, 3, 4, 5, 6]

In [10]:
# Create the param grid
param_grid = dict(n_neighbors = n_neighbors,
                  p = p)

print('KNeighbors Classifier')
knc_grid_search = GridSearchCV(knc, param_grid, scoring = 'accuracy', n_jobs = -1, cv = kfold, verbose = 1)
knc_grid_result = knc_grid_search.fit(X, Y)

dill.dump_session('parameter_testing.db')

# Updating the param grid
param_grid = dict(n_estimators = n_estimators)

print('AdaBoost Classifier')
abc_grid_search = GridSearchCV(abc, param_grid, scoring = 'accuracy', n_jobs = -1, cv = kfold, verbose = 1)
abc_grid_result = abc_grid_search.fit(X, Y)

dill.dump_session('parameter_testing.db')

# Updating the param grid
param_grid = dict(n_estimators = n_estimators,
                  max_depth = max_depth)

print('XGBoost Classifier')
xgb_grid_search = GridSearchCV(xgb, param_grid, scoring = 'accuracy', n_jobs = -1, cv = kfold, verbose = 1)
xgb_grid_result = xgb_grid_search.fit(X, Y)

dill.dump_session('parameter_testing.db')

# Updating the param grid
param_grid = dict(max_depth = max_depth,
                  min_samples_split = min_samples_split,
                  min_samples_leaf = min_samples_leaf)

print('DecisionTree Classifier')
dtc_grid_search = GridSearchCV(dtc, param_grid, scoring = 'accuracy', n_jobs = -1, cv = kfold, verbose = 1)
dtc_grid_result = dtc_grid_search.fit(X, Y)

dill.dump_session('parameter_testing.db')

# Updating the param grid
param_grid = dict(n_estimators = n_estimators,
                  max_depth = max_depth,
                  min_samples_split = min_samples_split,
                  min_samples_leaf = min_samples_leaf)

print('ExtraTrees Classifier')
etc_grid_search = GridSearchCV(etc, param_grid, scoring = 'accuracy', n_jobs = -1, cv = kfold, verbose = 1)
etc_grid_result = etc_grid_search.fit(X, Y)

dill.dump_session('parameter_testing.db')

print('RandomForest Classifier')
rfc_grid_search = GridSearchCV(rfc, param_grid, scoring = 'accuracy', n_jobs = -1, cv = kfold, verbose = 1)
rfc_grid_result = rfc_grid_search.fit(X, Y)

dill.dump_session('parameter_testing.db')

#print('GradientBoosting Classifier')
#gbc_grid_search = GridSearchCV(gbc, param_grid, scoring = 'accuracy', n_jobs = -1, cv = kfold, verbose = 1)
#gbc_grid_result = gbc_grid_search.fit(X, Y)

#dill.dump_session('parameter_testing.db')

KNeighbors Classifier
Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   33.3s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 19.7min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed: 27.2min finished


AdaBoost Classifier
Fitting 10 folds for each of 5 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   59.2s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  2.0min finished


XGBoost Classifier
Fitting 10 folds for each of 50 candidates, totalling 500 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 236 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 736 tasks      | elapsed:   19.9s
[Parallel(n_jobs=-1)]: Done 1436 tasks      | elapsed:   43.1s
[Parallel(n_jobs=-1)]: Done 2336 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 3000 out of 3000 | elapsed:  1.6min finished


ExtraTrees Classifier
Fitting 10 folds for each of 1500 candidates, totalling 15000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 200 tasks      | elapsed:   47.7s
[Parallel(n_jobs=-1)]: Done 450 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 800 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 1250 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 1800 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 2450 tasks      | elapsed: 10.3min
[Parallel(n_jobs=-1)]: Done 3200 tasks      | elapsed: 14.7min
[Parallel(n_jobs=-1)]: Done 4050 tasks      | elapsed: 19.1min
[Parallel(n_jobs=-1)]: Done 5000 tasks      | elapsed: 24.9min
[Parallel(n_jobs=-1)]: Done 6050 tasks      | elapsed: 33.3min
[Parallel(n_jobs=-1)]: Done 7200 tasks      | elapsed: 40.9min
[Parallel(n_jobs=-1)]: Done 8450 tasks      | elapsed: 49.2min
[Parallel(n_jobs=-1)]: Done 9800 tasks      | elapsed: 58.8min
[Parallel(n_jobs=-1)]: Done 11250 tasks      |

RandomForest Classifier
Fitting 10 folds for each of 1500 candidates, totalling 15000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed: 11.7min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed: 19.1min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed: 26.1min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed: 42.8min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed: 56.2min
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed: 77.8min
[Parallel(n_jobs=-1)]: Done 6034 tasks      | elapsed: 108.9min
[Parallel(n_jobs=-1)]: Done 7184 tasks      | elapsed: 132.9min
[Parallel(n_jobs=-1)]: Done 8434 tasks      | elapsed: 160.7min
[Parallel(n_jobs=-1)]: Done 9784 tasks      | elapsed: 192.9min
[Parallel(n_jobs=-1)]: Done 11234 tasks   

In [11]:
# Summarize results
print('KNeighbors Classifier:')
print("Best: %f using %s" % (knc_grid_result.best_score_, knc_grid_result.best_params_))

print('\nAdaBoost Classifier:')
print("Best: %f using %s" % (abc_grid_result.best_score_, abc_grid_result.best_params_))

print('\nXGBoost Classifier:')
print("Best: %f using %s" % (xgb_grid_result.best_score_, xgb_grid_result.best_params_))

print('\nDecisionTree Classifier:')
print("Best: %f using %s" % (dtc_grid_result.best_score_, dtc_grid_result.best_params_))

print('\nExtraTrees Classifier:')
print("Best: %f using %s" % (etc_grid_result.best_score_, etc_grid_result.best_params_))

print('\nRandomForest Classifier:')
print("Best: %f using %s" % (rfc_grid_result.best_score_, rfc_grid_result.best_params_))

#print('\nGradientBoosting Classifier:')
#print("Best: %f using %s" % (gbc_grid_result.best_score_, gbc_grid_result.best_params_))

KNeighbors Classifier:
Best: 0.852833 using {'n_neighbors': 24, 'p': 1.0}

AdaBoost Classifier:
Best: 0.999912 using {'n_estimators': 10}

XGBoost Classifier:
Best: 0.999956 using {'max_depth': 5, 'n_estimators': 10}

DecisionTree Classifier:
Best: 0.999956 using {'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 2}

ExtraTrees Classifier:
Best: 0.999978 using {'max_depth': 14, 'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 10}

RandomForest Classifier:
Best: 0.999956 using {'max_depth': 11, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 180}


In [12]:
'''
means = _grid_result.cv_results_['mean_test_score']
stds = _grid_result.cv_results_['std_test_score']
params = _grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
'''

'\nmeans = _grid_result.cv_results_[\'mean_test_score\']\nstds = _grid_result.cv_results_[\'std_test_score\']\nparams = _grid_result.cv_results_[\'params\']\nfor mean, stdev, param in zip(means, stds, params):\n    print("%f (%f) with: %r" % (mean, stdev, param))\n'